encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [1]:
# import sys
# sys.path.append('../../../../csi_transformer/src/')
# import backbone
# import image_segmentation
import numpy as np
import os
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 8 # 量化比特数

In [3]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"
ncodebooks = 32 # max:512
ncentroids = 256

train_sam_num = 150 # 训练集样本数
test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    linearin_path_train= ''
    linearout_path_train= ''
    linearin_path_test = ''
    linearout_path_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_joined = '/data/hdr/transformer_data/joined'
    dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    linearin_path_train= '%sin_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    y_train = '%s_y_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearout_path_train= '%sout_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearin_path_test = '%sin_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
    linearout_path_test = '%sout_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
else:
    raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


weightpath = '%s_w_f%i.npy' % (linear_name_full, feedback_bits)
biaspath = '%s_b_f%i.npy' % (linear_name_full, feedback_bits)
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [5]:
dataset_prepare(dir_joined, linear_name_full, feedback_bits, [train_sam_num, test_sam_num], batch_size, S1 = S1_dict[linear_name])

In [6]:
est3 = mm.estFactory(X_path=linearin_path_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)

running method:  Mithral
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    18753.59989278329
learn_multisplits(): returning loss:  381.7248059574049
learn_multisplits(): initial loss:    28489.33439981348
learn_multisplits(): returning loss:  974.4537388905883
learn_multisplits(): initial loss:    298.7503699480684
learn_multisplits(): returning loss:  72.5439866623256
learn_multisplits(): initial loss:    10141.544971659741
learn_multisplits(): returning loss:  262.61814920208417
learn_multisplits(): initial loss:    23556.00770876478
learn_multisplits(): returning loss:  830.3013631533831
learn_multisplits(): initial loss:    72355.41897514262
learn_multisplits(): returning loss:  1526.5472004413605
learn_multisplits(): initial loss:    655.6902937822678
learn_multisplits(): returning loss:  146.9239067180315
learn_multisplits(): initial loss:    18411.96517800063
learn_multisplits(): returning loss:  196.0421977280639
learn_multisplits(): initial loss:

In [7]:
x_test = np.load(dir_test+'/'+linearin_path_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)

[[ 0.47736773  0.19505306  0.44643864 ...  0.20700936  0.02614694
   0.40522146]
 [-0.02361076 -0.30592543  0.44643864 ...  0.20700936 -0.22434232
   0.1547322 ]
 [-0.02361076 -0.30592543  0.44643864 ... -0.29396912 -0.47483155
   0.40522146]
 ...
 [ 0.72785693 -0.05543617  0.19594939 ...  0.20700936 -0.22434232
   0.40522146]
 [ 0.72785693 -0.05543617  0.19594939 ...  0.20700936  0.02614694
   0.40522146]
 [ 0.47736773 -0.05543617  0.44643864 ...  0.20700936  0.02614694
   0.40522146]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
